In [63]:
import numpy as np
import pandas as pd
from PIL import Image

In [64]:
import torch
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize


In [65]:
from tqdm import tqdm

In [66]:
TRAIN_PATH = './input/data/train/data.csv'

In [67]:
class Custom_Dataset(Dataset):
    def __init__(self, path, target, transforms, train=True):
        self.train = train
        self.path = path
        self.transform = transforms
        
        self.data = pd.read_csv(path, index_col=0)
        self.classes = np.sort(self.data[target].unique())
        
        self.y = self.data[target]
        self.X = []
        for path in self.data['path']:
            im = Image.open(path)
            self.X.append(im)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        X, y = self.X[idx], self.y[idx]
        if self.transform:
          X = self.transform(X)
        return X, torch.tensor(y, dtype=torch.long)

In [68]:
transform = transforms.Compose([
    transforms.CenterCrop(250),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [69]:
dataset = Custom_Dataset(path=TRAIN_PATH, target='class', transforms=transform, train=True)

In [62]:
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4)

In [70]:
from torchvision import models

In [71]:
resnet50 = models.resnet50(pretrained=True)

In [72]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is using!')

cuda:0 is using!


In [73]:
resnet50.to(device)

lr_rate = 1e-3
epochs = 10

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet50.parameters(), lr=lr_rate)

In [75]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

for epoch in range(epochs):
    running_loss = 0
    running_acc = 0
    for index, (image, label) in enumerate(tqdm(dataloader)):
        image = image.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        logits = resnet50(image)
        _, preds = torch.max(logits, 1)
        loss = loss_fn(logits, label)
        loss.backward()
        optimizer.step()
    
        running_loss += loss.item() * image.size(0)
        running_acc += torch.sum(preds == label.data)
    
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_acc / len(dataloader.dataset)

    print(f'epoch : {epoch}, loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc}')


  0%|          | 0/296 [00:00<?, ?it/s]

epoch : 0, loss : 1.781, 평균 Accuracy : 0.4634920656681061


<ipython-input-42-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-42-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-42-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-42-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Py

epoch : 1, loss : 1.996, 평균 Accuracy : 0.38962963223457336


<ipython-input-42-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-42-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-42-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-42-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Py

...

epoch : 7, loss : 1.767, 평균 Accuracy : 0.44544973969459534


<ipython-input-42-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-42-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-42-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-42-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Py

epoch : 8, loss : 1.736, 평균 Accuracy : 0.45312169194221497


<ipython-input-42-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-42-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-42-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-42-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Py

epoch : 9, loss : 1.707, 평균 Accuracy : 0.4600529372692108


center crop : 250

batch_size = 64

epoch : 9, loss : 1.707, 평균 Accuracy : 0.4600529372692108